In [ ]:
# Studying the trade-off between exploration and exploitation
# The agent tries to learn the optimal policy in a maze
# The maze has two-way portals in it... See maze.py for attributions.

%matplotlib inline
import gym
import gym_maze
from maze import simulate, average_win_episode, new_maze, plot_episodes_against_parameter
import numpy as np

In [ ]:
# You can teach the agent to navigate the maze using different methods for exploration,
# Epsilon Greedy (EG), Decaying-EG (ED) or Upper Confidence Bounds (UCB)

# The usage for each parameter is documented in maze.py
params = {
    "winning_streak": 80,
    "policy": "EG",
    "epsilon": 0.2,
    "learning_rate": 1.0,
    "starting_value": 0.0,
    "display": False
}

winning_episode = simulate(maze=new_maze(), n_episodes=1000, **params)["winning_episode"] 
print(f"The maze was learnt in {winning_episode} episodes.")

In [ ]:
# Increase these values for better plots!
n_mazes = 100
n_levels = 10

# *--- EPSILON-GREEDY (EG) ---*
# Best initialization value for Epsilon-Greedy
initial_values = np.linspace(-1.0, 2.0, int(n_levels/3.0))
plot_y = plot_episodes_against_parameter("starting_value", initial_values, 
                                               params, n_mazes=n_mazes)

# Check whether it's beneficial to use optimistic initialization. 
# Does that change if epsilon (exploration) is 0.0?

In [ ]:
# Let's choose the best 'starting_value'
params["starting_value"] = initial_values[np.argmin(plot_y)]

# Best Epsilon for Epsilon-Greedy (with best initialization value)
params["learning_rate"] = 1.0
epsilons = np.linspace(0.0, 0.5, n_levels)
plot_y = plot_episodes_against_parameter("epsilon", epsilons, 
                                               params, n_mazes=n_mazes)

In [ ]:
# Let's choose the best 'epsilon'
params["epsilon"] = epsilons[np.argmin(plot_y)]

# Best Learning-Rate for Epsilon-Greedy
learning_rates = np.linspace(0.001, 1.5, n_levels)
plot_y = plot_episodes_against_parameter("learning_rate", learning_rates, 
                                               params, n_mazes=n_mazes)

In [ ]:
# *--- DECAYING EPSILON-GREEDY (ED) ---*
params["policy"] = "ED"
params["epsilon"] = 0.3
params["learning_rate"] = 1.15
params["starting_value"] = 0.3

n_mazes_small = int(n_mazes/5)
# Best decay-factor and learning rate (3-step optimization)
# First optimize decay-factor
decay_factors = np.linspace(0.0, 1.0, n_levels)
plot_y = plot_episodes_against_parameter("decay", decay_factors, params, n_mazes=n_mazes_small, view_plot=False)
params["decay"] = decay_factors[np.argmin(plot_y)]

# Then the learning rate
learning_rates = np.linspace(0.1, 1.5, n_levels)
plot_y = plot_episodes_against_parameter("learning_rate", learning_rates, params, n_mazes=n_mazes_small, view_plot=False)
params["learning_rate"] = learning_rates[np.argmin(plot_y)]

# Then the epsilon
epsilons = np.linspace(0.1, 1.1, n_levels)
plot_y = plot_episodes_against_parameter("learning_rate", learning_rates, params, n_mazes=n_mazes_small, view_plot=False)
params["learning_rate"] = learning_rates[np.argmin(plot_y)]

# Then the decay factor
decay_factors = np.linspace(0.0, 1.0, n_levels)
plot_y = plot_episodes_against_parameter("decay", decay_factors, params, n_mazes=n_mazes)

In [ ]:
# *--- UPPER CONFIDENCE BOUNDS (UCB) ---*
params["policy"] = "UCB"
params["epsilon"] = 0.1
params["learning_rate"] = 0.5
params["starting_value"] = 0.0

# Best decay-factor (UCB values decay everytime a state-action is visited)
decay_factors = np.linspace(0.1, 1.0, n_levels)
plot_y = plot_episodes_against_parameter("decay", decay_factors, 
                                               params, n_mazes=n_mazes)